# **1. Perkenalan Dataset**


### **A. Deskripsi Dataset**
Dataset yang digunakan dalam proyek ini adalah data transaksi **Online Retail**. Dataset ini merekam jejak transaksi penjualan antar-negara (*cross-border*) dari sebuah perusahaan ritel daring yang berbasis di Inggris (UK). Data ini mencakup seluruh transaksi yang terjadi dalam rentang waktu **01 Desember 2010 hingga 09 Desember 2011**.

Perusahaan ini berfokus pada penjualan hadiah unik untuk berbagai kesempatan, dengan basis pelanggan utama yang terdiri dari pedagang grosir (*wholesaler*). Data mentah yang tersedia memiliki format *time-series transactional* dengan atribut sebagai berikut:

* **`InvoiceNo`**: Nomor unik (integer/string) yang ditetapkan untuk setiap transaksi. Jika kode dimulai dengan 'c', itu menandakan pembatalan.
* **`StockCode`**: Kode unik (string) untuk setiap jenis produk yang berbeda.
* **`Description`**: Deskripsi teks atau nama dari produk.
* **`Quantity`**: Jumlah unit produk yang dibeli per transaksi.
* **`InvoiceDate`**: Tanggal dan waktu (timestamp) terjadinya transaksi.
* **`UnitPrice`**: Harga per unit produk (dalam mata uang Sterling).
* **`CustomerID`**: Nomor unik (integer) yang ditetapkan untuk setiap pelanggan.
* **`Country`**: Nama negara tempat pelanggan bertransaksi.

---

### **B. Masalah Bisnis & Rancangan Model**
Tujuan utama dari proyek ini adalah membangun sistem **Machine Learning Klasifikasi (Multi-Class Classification)** untuk mengelompokkan produk ke dalam kategori performa penjualan. Pengelompokan ini bertujuan untuk membantu tim bisnis dalam manajemen inventaris, strategi penetapan harga, dan pemasaran produk.

#### **Tantangan Data**
Dataset asli bersifat *unsupervised* (tidak memiliki label kategori) dan berada pada level *transaksi*. Oleh karena itu, model tidak dapat dilatih langsung menggunakan data mentah.

#### **Strategi Pemodelan (Pipeline)**
Untuk mengatasi tantangan tersebut, dirancang alur pemrosesan data sebagai berikut:

1.  **Transformasi Data (Transaction to Product Level)**
    Data akan dikelompokkan berdasarkan `StockCode` untuk mendapatkan metrik performa per produk, bukan per transaksi.

2.  **Feature Engineering (Ekstraksi Fitur)**
    Kita akan mengekstrak indikator statistik utama untuk setiap produk:
    * **`Avg_Sales` (Volume):** Rata-rata penjualan per bulan.
    * **`Std_Dev` (Volatilitas):** Seberapa fluktuatif penjualan produk tersebut.
    * **`Max_Sales` (Potensi):** Jumlah penjualan tertinggi yang pernah dicapai dalam satu waktu.
    * **`CV` (Coefficient of Variation):** Rasio antara standar deviasi dan rata-rata. Ini adalah indikator **Kestabilan**. Semakin rendah nilai CV, semakin stabil penjualan produk tersebut.
    * **`Avg_Revenue` (Nilai Bisnis):** Estimasi pendapatan rata-rata yang dihasilkan produk (Quantity × UnitPrice).

3.  **Penentuan Label Otomatis (Rule-Based Labeling)**
    Karena data tidak memiliki label, kita akan membuat label sintetis menggunakan pendekatan **Quadrant Analysis** berdasarkan ambang batas statistik (*Statistical Thresholds*):
    * **Threshold Revenue:** Menggunakan *Quartile 3 (75%)* dari `Avg_Revenue`.
    * **Threshold Stabilitas:** Menggunakan *Median* dari `CV`.


---

### **c. Output Model**
#### **Definisi Kelas Target (Label 0-3)**
Model akan dilatih untuk memprediksi 4 kategori produk:

* **KELAS 3 (Star Product / VIP):**
    Produk dengan **Revenue Tinggi** (> Q3) dan **Penjualan Stabil** (CV Rendah). Ini adalah produk andalan perusahaan.
* **KELAS 2 (High Potential / Risky):**
    Produk dengan **Revenue Tinggi** (> Q3) namun **Tidak Stabil** (CV Tinggi). Produk ini laku keras sesekali (musiman/tren).
* **KELAS 1 (Consistent Learner):**
    Produk dengan **Revenue Sedang/Rendah** namun **Sangat Stabil**. Produk ini memberikan arus kas yang pasti meskipun nilainya kecil.
* **KELAS 0 (Low Performer):**
    Produk dengan **Revenue Rendah** dan **Tidak Stabil**. Produk yang jarang terjual dan nilainya kecil.

# **2. Import Library**

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **3. Memuat Dataset**

In [9]:
# Load data user
url='https://drive.google.com/uc?id=1l4OlQwoikH_mqFRJUZeJO64_pyr82oKg'
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# **4. Exploratory Data Analysis (EDA)**


In [10]:
print("Info Dataset:")
df.info()
print("\nStatistik Deskriptif:")
df.describe()

Info Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB

Statistik Deskriptif:


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


# **5. Data Preprocessing**

### 5.1: Cleaning Data

In [11]:
df_clean = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)].copy()
df_clean = df_clean[df_clean['Quantity'] < 10000]
df_clean.dropna(subset=['CustomerID'], inplace=True)
df_clean = df_clean.drop_duplicates()
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

print(f"Ukuran data setelah cleaning: {df_clean.shape}")

Ukuran data setelah cleaning: (392690, 8)


### 5.2: Feature Engineering (Membuat Fitur Produk)

In [12]:
df_clean['Month_Year'] = df_clean['InvoiceDate'].dt.to_period('M')

monthly_sales = df_clean.pivot_table(
    index='StockCode',
    columns='Month_Year',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

product_features = pd.DataFrame()
product_features['Avg_Sales'] = monthly_sales.mean(axis=1)
product_features['Std_Dev'] = monthly_sales.std(axis=1)
product_features['Max_Sales'] = monthly_sales.max(axis=1)

product_features['CV'] = product_features['Std_Dev'] / product_features['Avg_Sales']
product_features['CV'] = product_features['CV'].fillna(0)

product_info = df_clean.groupby('StockCode').agg({'UnitPrice': 'mean', 'Description': 'first'})
final_data = product_features.join(product_info)
final_data['Avg_Revenue'] = final_data['Avg_Sales'] * final_data['UnitPrice']

### 5.3: Labeling (Penentuan Kuadran)

In [13]:
REV_Q3 = final_data['Avg_Revenue'].quantile(0.75)
MAX_Q3 = final_data['Max_Sales'].quantile(0.75)
CV_MEDIAN = final_data['CV'].median()

print(f"Threshold Omzet Tinggi (Revenue) > {REV_Q3:.2f}")
print(f"Threshold Stabil (CV) < {CV_MEDIAN:.2f}")

def quadrant_labeling(row):
    revenue = row['Avg_Revenue']
    max_sales = row['Max_Sales']
    volatility = row['CV']

    is_high_revenue = (revenue > REV_Q3) or (max_sales > MAX_Q3)
    is_stable = (volatility <= CV_MEDIAN)

    if is_high_revenue and is_stable:
        return 3

    elif is_high_revenue:
        return 2

    elif is_stable:
        return 1

    else:
        return 0

final_data['Label'] = final_data.apply(quadrant_labeling, axis=1)

print("\nSebaran 4 Label Baru:")
print(final_data['Label'].value_counts().sort_index())

Threshold Omzet Tinggi (Revenue) > 172.19
Threshold Stabil (CV) < 1.39

Sebaran 4 Label Baru:
Label
0    1436
1     982
2     396
3     850
Name: count, dtype: int64


### 5.4: Menyimpan Hasil (Simpan Dataset Final)

In [14]:
final_data.reset_index().to_csv('data_training_final.csv', index=False)

In [15]:
final_data.head()

,Avg_Sales,Std_Dev,Max_Sales,CV,UnitPrice,Description,Avg_Revenue,Label
StockCode,,,,,,,,
10002,63.307692,111.949680,337,1.768342,0.850000,INFLATABLE POLITICAL GLOBE,53.811538,0
10080,22.384615,31.473636,91,1.406039,0.411905,GROOVY CACTUS INFLATABLE,9.220330,0
10120,14.769231,15.812199,49,1.070618,0.210000,DOGGY RUBBER,3.101538,1
10123C,0.384615,1.120897,4,2.914332,0.650000,HEARTS WRAPPING TAPE,0.250000,0
10124A,1.230769,1.964427,5,1.596097,0.420000,SPOTS ON RED BOOKCOVER TAPE,0.516923,0
